# Module 07: Bollinger Bands & Volatility Analysis

---

## Welcome to Intermediate Technical Analysis! 🎓

Congratulations on completing the beginner modules! You've built a solid foundation in technical analysis. Now we're ready to explore more sophisticated tools that professional traders use daily.

### 🎯 Learning Objectives

By the end of this module, you will be able to:
- ✅ Understand what volatility is and why it matters for trading
- ✅ Calculate and interpret Bollinger Bands
- ✅ Identify Bollinger Band squeeze and expansion patterns
- ✅ Use ATR (Average True Range) to measure volatility
- ✅ Compare Bollinger Bands with Keltner Channels
- ✅ Set volatility-based stop-loss levels
- ✅ Combine volatility indicators with trend analysis

### Prerequisites

Before starting this module, you should have completed:
- ✅ Modules 00-06 (Beginner series)
- ✅ Understanding of moving averages (SMA/EMA)
- ✅ Familiarity with standard deviation concept
- ✅ Comfort with creating charts in Python

###  Time Required

60-75 minutes

---

## Part 1: Understanding Volatility

### What is Volatility?

**Volatility** measures how much and how quickly a stock's price changes. It's one of the most important concepts in trading!

### 🎯 Why Volatility Matters

Think of volatility like weather patterns:
- **Low Volatility** = Calm, predictable weather (gentle price movements)
- **High Volatility** = Stormy, unpredictable weather (wild price swings)

**For Traders:**
1. **Risk Assessment**: High volatility = higher risk (and potentially higher reward)
2. **Stop-Loss Placement**: Volatile stocks need wider stop-losses
3. **Position Sizing**: Should risk less on volatile stocks
4. **Strategy Selection**: Different strategies work in different volatility environments

### Real-World Example

**Low Volatility Stock (Maybank - Blue Chip):**
- Monday: RM 9.00
- Tuesday: RM 9.05
- Wednesday: RM 9.03
- Thursday: RM 9.07
- Friday: RM 9.04
- *Daily change: ~0.5%* → Predictable, safer

**High Volatility Stock (Small Cap Tech):**
- Monday: RM 1.00
- Tuesday: RM 1.15 (+15%!)
- Wednesday: RM 0.95 (-17%!)
- Thursday: RM 1.08 (+13%!)
- Friday: RM 0.98 (-9%!)
- *Daily change: ~10-15%* → Unpredictable, risky

### 💡 Key Insight

Volatility is **neither good nor bad** - it's simply information!
- High volatility = More opportunity (and more risk)
- Low volatility = Less opportunity (and less risk)
- **The key is matching your strategy to the volatility environment!**

---

## Part 2: Setting Up Our Environment

Let's import our libraries and fetch data for UUE Holdings (our case study stock).

In [ ]:
# Import required libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas_ta as ta
import warnings

warnings.filterwarnings("ignore")

# Set up plotting style
%matplotlib inline
try:
    plt.style.use("seaborn-v0_8-darkgrid")
except:
    try:
        plt.style.use("seaborn-darkgrid")
    except:
        plt.style.use("default")

sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print(f"📅 Ready to analyze volatility!")

### Fetch Stock Data

We'll get 1 year of data for UUE Holdings to have enough history for volatility analysis.

In [ ]:
# Fetch 1 year of data for UUE Holdings Berhad
ticker = "0310.KL"  # UUE Holdings
print(f"Fetching data for {ticker}...\\n")

try:
    data = yf.download(ticker, period="1y", progress=False)

    # Flatten column names if MultiIndex (for single ticker downloads)
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)

    if data.empty:
        print("No data received. Try again or use a different ticker.")
    else:
        print(f"Successfully fetched {len(data)} days of data!")
        print(f"Date range: {data.index[0].date()} to {data.index[-1].date()}")
        print(f"Latest close: RM {data['Close'].iloc[-1]:.2f}\\n")

        # Display first few rows
        print("First 5 days:")
        print(data.head())

except Exception as e:
    print(f"Error: {e}")
    print("Try checking your internet connection or using a different ticker.")

---

## Part 3: Bollinger Bands - Theory

### What are Bollinger Bands?

**Bollinger Bands** were created by John Bollinger in the 1980s. They consist of **three lines**:

1. **Middle Band** = 20-day Simple Moving Average (SMA)
2. **Upper Band** = Middle Band + (2 × Standard Deviation)
3. **Lower Band** = Middle Band - (2 × Standard Deviation)

### 🎯 What Do They Tell Us?

Bollinger Bands create a **dynamic envelope** around price that:
- **Expands** during high volatility (bands widen)
- **Contracts** during low volatility (bands narrow)
- Shows **overbought** conditions when price near upper band
- Shows **oversold** conditions when price near lower band

### Real-World Analogy

Think of Bollinger Bands like a **rubber band**:
- When price touches the upper band → rubber band stretched UP (may snap back)
- When price touches the lower band → rubber band stretched DOWN (may snap back)
- When bands are narrow → rubber band relaxed (big move coming soon!)
- When bands are wide → rubber band stretched (volatility high, may calm down)

### The 20/2 Standard

**Default Settings: 20-period SMA with 2 standard deviations**

Why these numbers?
- **20 periods** = Roughly one trading month (20-22 trading days)
- **2 standard deviations** = Statistically, ~95% of price action stays within the bands
- **When price goes outside** = Statistically rare event (potential reversal!)

### 💡 Key Principle

**"What goes up must come down, and what goes down must come up"** - BUT NOT IMMEDIATELY!

Bollinger Bands help us identify:
1. When price has moved too far, too fast (stretched bands)
2. When a big move is about to happen (squeezed bands)
3. The general trend direction (price position relative to middle band)

---

## Part 4: Calculating Bollinger Bands

Let's calculate Bollinger Bands for UUE Holdings using pandas-ta.

In [ ]:
# Calculate Bollinger Bands manually
# Default: 20-period SMA with 2 standard deviations

if "data" in locals() and not data.empty:
    # Calculate components
    period = 20
    std_dev = 2

    # Middle Band = 20-day Simple Moving Average
    data["BBM_20_2.0"] = data["Close"].rolling(window=period).mean()

    # Standard Deviation
    rolling_std = data["Close"].rolling(window=period).std()

    # Upper Band = Middle + (2 × StdDev)
    data["BBU_20_2.0"] = data["BBM_20_2.0"] + (std_dev * rolling_std)

    # Lower Band = Middle - (2 × StdDev)
    data["BBL_20_2.0"] = data["BBM_20_2.0"] - (std_dev * rolling_std)

    # Band Width (as percentage)
    data["BBB_20_2.0"] = (data["BBU_20_2.0"] - data["BBL_20_2.0"]) / data["BBM_20_2.0"]

    # %B - Where price is within the bands (0 = lower, 1 = upper, 0.5 = middle)
    data["BBP_20_2.0"] = (data["Close"] - data["BBL_20_2.0"]) / (
        data["BBU_20_2.0"] - data["BBL_20_2.0"]
    )

    print("Bollinger Bands calculated successfully!\\n")
    print("Last 5 days with Bollinger Bands:")
    print(data[["Close", "BBL_20_2.0", "BBM_20_2.0", "BBU_20_2.0"]].tail())

    # Show current position
    latest_close = data["Close"].iloc[-1]
    latest_upper = data["BBU_20_2.0"].iloc[-1]
    latest_middle = data["BBM_20_2.0"].iloc[-1]
    latest_lower = data["BBL_20_2.0"].iloc[-1]
    latest_pct_b = data["BBP_20_2.0"].iloc[-1]

    print("\\nCurrent Position:")
    print(f"Price: RM {latest_close:.2f}")
    print(f"Upper Band: RM {latest_upper:.2f}")
    print(f"Middle Band: RM {latest_middle:.2f}")
    print(f"Lower Band: RM {latest_lower:.2f}")
    print(f"%B: {latest_pct_b:.2f} (0 = at lower band, 0.5 = at middle, 1 = at upper band)")

    # Interpretation
    if latest_pct_b > 0.8:
        print("\\nPrice is near upper band - Potentially overbought!")
    elif latest_pct_b < 0.2:
        print("\\nPrice is near lower band - Potentially oversold!")
    else:
        print("\\nPrice is in the middle range - Normal trading")
else:
    print("Please run the data fetching cell first!")

### Visualizing Bollinger Bands

Let's create a chart showing price with Bollinger Bands overlaid.

In [ ]:
# Create Bollinger Bands chart
if "data" in locals() and not data.empty and "BBU_20_2.0" in data.columns:
    # Use last 6 months for clearer visualization
    recent_data = data.tail(120)

    plt.figure(figsize=(15, 8))

    # Plot Bollinger Bands
    plt.plot(
        recent_data.index,
        recent_data["Close"],
        label="Close Price",
        linewidth=2,
        color="black",
        zorder=5,
    )
    plt.plot(
        recent_data.index,
        recent_data["BBU_20_2.0"],
        label="Upper Band",
        linestyle="--",
        color="red",
        alpha=0.7,
    )
    plt.plot(
        recent_data.index,
        recent_data["BBM_20_2.0"],
        label="Middle Band (20 SMA)",
        linestyle="--",
        color="blue",
        alpha=0.7,
    )
    plt.plot(
        recent_data.index,
        recent_data["BBL_20_2.0"],
        label="Lower Band",
        linestyle="--",
        color="green",
        alpha=0.7,
    )

    # Fill area between bands
    plt.fill_between(
        recent_data.index,
        recent_data["BBU_20_2.0"],
        recent_data["BBL_20_2.0"],
        alpha=0.1,
        color="gray",
    )

    plt.title(f"{ticker} - Bollinger Bands (20, 2)", fontsize=16, fontweight="bold")
    plt.xlabel("Date", fontsize=12)
    plt.ylabel("Price (RM)", fontsize=12)
    plt.legend(loc="best", fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    print("\\n\\nChart Explanation:")
    print("- Gray shaded area = Bollinger Bands envelope")
    print("- When price touches RED line = Near upper band (potentially overbought)")
    print("- When price touches GREEN line = Near lower band (potentially oversold)")
    print("- Blue dashed line = 20-day moving average (middle band)")
    print("- Wide bands = High volatility period")
    print("- Narrow bands = Low volatility period (squeeze!)")
else:
    print("Please run the Bollinger Bands calculation cell first!")

---

## Part 5: The Bollinger Band Squeeze

### What is a Squeeze?

A **Bollinger Band Squeeze** occurs when the bands contract to their narrowest width. This indicates:
- Very low volatility
- Market consolidation
- A big move is likely coming soon!

### Why Squeezes Matter

Think of a squeeze like a **coiled spring**:
- The tighter the squeeze → The bigger the potential release
- Markets alternate between low volatility (squeeze) and high volatility (expansion)
- Traders position themselves BEFORE the breakout

### How to Identify a Squeeze

1. **Band Width Indicator**: When BB Width is at multi-month lows
2. **Visual**: Bands look very narrow on the chart
3. **%B Indicator**: Price oscillating between 0.3 and 0.7 (stuck in middle)

### Trading the Squeeze

**Strategy**:
1. Identify when bands are at their narrowest (squeeze)
2. Wait for breakout (price closes outside the bands)
3. Enter in direction of breakout
4. Place stop-loss on opposite band

**Example**:
- Squeeze identified (narrow bands)
- Price breaks ABOVE upper band → Buy signal
- Stop-loss at lower band
- Expect expansion and trend continuation

In [ ]:
# Detect Bollinger Band Squeezes
if "data" in locals() and not data.empty and "BB_Width" in data.columns:
    # Calculate the rolling minimum of band width (last 6 months)
    data["BB_Width_Min_6M"] = data["BB_Width"].rolling(window=125).min()

    # A squeeze is when current width is within 10% of 6-month minimum
    data["Is_Squeeze"] = data["BB_Width"] <= (data["BB_Width_Min_6M"] * 1.1)

    # Find recent squeezes
    recent_squeezes = data[data["Is_Squeeze"]].tail(10)

    print("Bollinger Band Squeeze Detection\\n")
    print("=" * 60)

    if len(recent_squeezes) > 0:
        print(f"\\n\\nFound {len(recent_squeezes)} squeeze days in recent history:\\n")
        print(recent_squeezes[["Close", "BB_Width", "BB_Percent"]].to_string())

        # Check if currently in squeeze
        if data["Is_Squeeze"].iloc[-1]:
            print("\\n\\n*** CURRENTLY IN A SQUEEZE! ***")
            print("Watch for a breakout - big move may be coming!")
            print(f"Current Band Width: {data['BB_Width'].iloc[-1]:.4f}")
            print(f"6-Month Min Width: {data['BB_Width_Min_6M'].iloc[-1]:.4f}")
        else:
            print("\\n\\nNot currently in a squeeze.")
            print(f"Current Band Width: {data['BB_Width'].iloc[-1]:.4f}")
            print(f"6-Month Min Width: {data['BB_Width_Min_6M'].iloc[-1]:.4f}")
    else:
        print("No significant squeezes detected in recent data.")
else:
    print("Please run the Bollinger Bands calculation cell first!")

# Detect Bollinger Band Squeezes
if 'data' in locals() and not data.empty and 'BBB_20_2.0' in data.columns:
    # Calculate the rolling minimum of band width (last 6 months)
    data['BB_Width_Min_6M'] = data['BBB_20_2.0'].rolling(window=125).min()
    
    # A squeeze is when current width is within 10% of 6-month minimum
    data['Is_Squeeze'] = data['BBB_20_2.0'] <= (data['BB_Width_Min_6M'] * 1.1)
    
    # Find recent squeezes
    recent_squeezes = data[data['Is_Squeeze']].tail(10)
    
    print("Bollinger Band Squeeze Detection\\n")
    print("="*60)
    
    if len(recent_squeezes) > 0:
        print(f"\\n\\nFound {len(recent_squeezes)} squeeze days in recent history:\\n")
        print(recent_squeezes[['Close', 'BBB_20_2.0', 'BBP_20_2.0']].to_string())
        
        # Check if currently in squeeze
        if data['Is_Squeeze'].iloc[-1]:
            print("\\n\\n*** CURRENTLY IN A SQUEEZE! ***")
            print("Watch for a breakout - big move may be coming!")
            print(f"Current Band Width: {data['BBB_20_2.0'].iloc[-1]:.4f}")
            print(f"6-Month Min Width: {data['BB_Width_Min_6M'].iloc[-1]:.4f}")
        else:
            print("\\n\\nNot currently in a squeeze.")
            print(f"Current Band Width: {data['BBB_20_2.0'].iloc[-1]:.4f}")
            print(f"6-Month Min Width: {data['BB_Width_Min_6M'].iloc[-1]:.4f}")
    else:
        print("No significant squeezes detected in recent data.")
else:
    print("Please run the Bollinger Bands calculation cell first!")

In [ ]:
# Calculate ATR using pandas-ta
if "data" in locals() and not data.empty:
    # Calculate ATR (14-period standard)
    data["ATR"] = ta.atr(data["High"], data["Low"], data["Close"], length=14)

    print("ATR (Average True Range) Analysis\n")
    print("=" * 60)

    # Show recent ATR values
    print("\nLast 10 days - ATR values:")
    print(data[["Close", "High", "Low", "ATR"]].tail(10).to_string())

    # Current ATR analysis
    current_atr = data["ATR"].iloc[-1]
    current_price = data["Close"].iloc[-1]
    atr_percentage = (current_atr / current_price) * 100

    print(f"\nCurrent ATR Analysis:")
    print(f"Price: RM {current_price:.2f}")
    print(f"ATR: RM {current_atr:.3f}")
    print(f"ATR as % of price: {atr_percentage:.2f}%")

    # ATR-based stop-loss suggestion
    stop_loss_2x = current_price - (2 * current_atr)

    print(f"\nSuggested Stop-Loss (2 x ATR): RM {stop_loss_2x:.2f}")

    # Volatility assessment
    print(f"\nVolatility Assessment:")
    if atr_percentage < 2:
        print(f"LOW volatility ({atr_percentage:.2f}%) - Calm movements")
    elif atr_percentage < 5:
        print(f"MODERATE volatility ({atr_percentage:.2f}%) - Normal trading")
    else:
        print(f"HIGH volatility ({atr_percentage:.2f}%) - Wild swings!")
else:
    print("Please run the data fetching cell first!")

---

## Part 7: Trading Strategies with Bollinger Bands & ATR

### Strategy 1: Bollinger Bounce

**Concept**: Buy at lower band, sell at upper band in ranging markets

**Rules**:
1. Wait for price to touch/pierce lower band
2. Wait for reversal signal (bullish candle, RSI oversold)
3. Enter long position
4. Target: Upper band or middle band
5. Stop-loss: 2 × ATR below entry

**Best for**: Sideways/ranging markets

**Example**:
- Price touches RM 1.00 (lower band)
- RSI = 25 (oversold)
- Enter long at RM 1.02
- Target: RM 1.20 (upper band)
- Stop-loss: RM 1.02 - (2 × 0.05 ATR) = RM 0.92

### Strategy 2: Squeeze Breakout

**Concept**: Trade the expansion after a squeeze

**Rules**:
1. Identify Bollinger Band squeeze (bands very narrow)
2. Wait for price to break above/below bands
3. Enter in direction of breakout
4. Place stop at opposite band
5. Exit when opposite band is touched OR 2 × ATR profit taken

**Best for**: Anticipating big moves after consolidation

**Example**:
- Squeeze identified (band width at 6-month low)
- Price breaks ABOVE upper band at RM 1.15
- Enter long
- Stop-loss: RM 1.05 (lower band)
- Target: Current price + (2 × ATR) = RM 1.15 + RM 0.10 = RM 1.25

### Strategy 3: Trend Following with BB + ATR

**Concept**: Use BB for trend, ATR for stop-loss

**Rules**:
1. **Uptrend**: Price stays above middle band (20 SMA)
2. Buy on pullbacks to middle band
3. Stop-loss: 2 × ATR below entry
4. Exit: Price closes below middle band OR hits profit target
5. Target: 3 × ATR or previous resistance

**Best for**: Trending markets

**Example - Uptrend**:
- Price above middle band (RM 1.10)
- Pullback to middle band
- Enter long at RM 1.11
- ATR = RM 0.05
- Stop: RM 1.11 - (2 × 0.05) = RM 1.01
- Target: RM 1.11 + (3 × 0.05) = RM 1.26

In [ ]:
# Let's implement a simple Bollinger Bounce signal detector
if "data" in locals() and not data.empty and "BB_Lower" in data.columns:
    # Detect potential bounce setups
    data["Touch_Lower"] = data["Low"] <= data["BB_Lower"]
    data["Touch_Upper"] = data["High"] >= data["BB_Upper"]

    # Find recent touches
    recent_lower_touches = data[data["Touch_Lower"]].tail(5)
    recent_upper_touches = data[data["Touch_Upper"]].tail(5)

    print("Bollinger Band Touch Analysis\n")
    print("=" * 60)

    print("\nRecent LOWER Band Touches (Potential Buy Opportunities):")
    if len(recent_lower_touches) > 0:
        print(recent_lower_touches[["Close", "BB_Lower", "BB_Percent"]].to_string())
    else:
        print("No recent lower band touches")

    print("\nRecent UPPER Band Touches (Potential Sell Opportunities):")
    if len(recent_upper_touches) > 0:
        print(recent_upper_touches[["Close", "BB_Upper", "BB_Percent"]].to_string())
    else:
        print("No recent upper band touches")

    # Current setup
    print("\n" + "=" * 60)
    print("CURRENT SETUP:")
    current_pct_b = data["BB_Percent"].iloc[-1]

    if current_pct_b < 0.2:
        print("POTENTIAL BUY SETUP - Price near lower band!")
        print(f"Consider buying if reversal confirmed")
        print(f"Target: Middle band at RM {data['BB_Middle'].iloc[-1]:.2f}")
    elif current_pct_b > 0.8:
        print("POTENTIAL SELL SETUP - Price near upper band!")
        print(f"Consider selling/taking profits")
        print(f"Target: Middle band at RM {data['BB_Middle'].iloc[-1]:.2f}")
    else:
        print("NO SETUP - Price in middle range")
        print(f"Wait for price to reach extremes")
else:
    print("Please run the Bollinger Bands calculation cell first!")

# Let's implement a simple Bollinger Bounce signal detector
if 'data' in locals() and not data.empty and 'BBL_20_2.0' in data.columns:
    # Detect potential bounce setups
    data['Touch_Lower'] = data['Low'] <= data['BBL_20_2.0']
    data['Touch_Upper'] = data['High'] >= data['BBU_20_2.0']

    # Find recent touches
    recent_lower_touches = data[data['Touch_Lower']].tail(5)
    recent_upper_touches = data[data['Touch_Upper']].tail(5)

    print("Bollinger Band Touch Analysis\\n")
    print("="*60)

    print("\\nRecent LOWER Band Touches (Potential Buy Opportunities):")
    if len(recent_lower_touches) > 0:
        print(recent_lower_touches[['Close', 'BBL_20_2.0', 'BBP_20_2.0']].to_string())
    else:
        print("No recent lower band touches")

    print("\\nRecent UPPER Band Touches (Potential Sell Opportunities):")
    if len(recent_upper_touches) > 0:
        print(recent_upper_touches[['Close', 'BBU_20_2.0', 'BBP_20_2.0']].to_string())
    else:
        print("No recent upper band touches")

    # Current setup
    print("\\n" + "="*60)
    print("CURRENT SETUP:")
    current_pct_b = data['BBP_20_2.0'].iloc[-1]

    if current_pct_b < 0.2:
        print("POTENTIAL BUY SETUP - Price near lower band!")
        print(f"Consider buying if reversal confirmed")
        print(f"Target: Middle band at RM {data['BBM_20_2.0'].iloc[-1]:.2f}")
    elif current_pct_b > 0.8:
        print("POTENTIAL SELL SETUP - Price near upper band!")
        print(f"Consider selling/taking profits")
        print(f"Target: Middle band at RM {data['BBM_20_2.0'].iloc[-1]:.2f}")
    else:
        print("NO SETUP - Price in middle range")
        print(f"Wait for price to reach extremes")
else:
    print("Please run the Bollinger Bands calculation cell first!")

---

## Practice Exercises

### Exercise 1: Identify Squeezes
1. Run the squeeze detection code
2. Find the dates when UUE had the tightest bands
3. What happened to price after those squeezes?
4. Did price break up or down?

**Reflection**: How long did squeezes typically last before breakout?

### Exercise 2: Try Different Stocks
Modify the ticker variable and analyze:
- **1155.KL** (Maybank - large cap, low volatility)
- **5296.KL** (Tenaga - utility, moderate volatility)

**Questions**:
1. Which stock has wider Bollinger Bands?
2. Which has higher ATR?
3. Which would you need wider stop-losses for?

In [ ]:
# Exercise 2: Try different stocks
# Uncomment and run:

# ticker = "1155.KL"  # Maybank
# data = yf.download(ticker, period="1y", progress=False)
# # Then re-run the BB and ATR calculations

### Exercise 3: Calculate Position Size

**Scenario**:
- You have RM 10,000 capital
- You want to risk 2% per trade (RM 200)
- UUE is at RM 1.20
- ATR is RM 0.06
- You'll use 2 × ATR stop-loss

**Questions**:
1. What's your stop-loss price?
2. What's your risk per share?
3. How many shares can you buy?

**Solution**:
1. Stop-loss = 1.20 - (2 × 0.06) = RM 1.08
2. Risk per share = 1.20 - 1.08 = RM 0.12
3. Shares = 200 / 0.12 = 1,666 shares

---

## Common Mistakes to Avoid

### 1. Using BB in Strong Trends
**Mistake**: Shorting at upper band during strong uptrend
**Why it fails**: In trends, price can "walk the band" for extended periods
**Solution**: Use BB with trend indicators (above middle = uptrend)

### 2. Ignoring ATR When Setting Stops
**Mistake**: Using fixed percentage stops (always 5%)
**Why it fails**: Doesn't account for stock's volatility
**Solution**: Use ATR-based stops (2 × ATR)

### 3. Trading Every Band Touch
**Mistake**: Buying every lower band touch automatically
**Why it fails**: Not all touches result in bounces
**Solution**: Wait for confirmation (bullish candle, RSI oversold, volume)

### 4. Wrong Squeeze Interpretation
**Mistake**: Buying during squeeze before breakout
**Why it fails**: Don't know which direction it will break
**Solution**: Wait for breakout THEN trade in that direction

### 5. Forgetting the Market Environment
**Mistake**: Using bounce strategy in trending market
**Why it fails**: Different strategies for different environments
**Solution**:
- Ranging market → Bounce strategy
- Trending market → Trend following strategy
- Squeeze → Breakout strategy

---

## What's Next?

### You've Completed Module 07!

Congratulations! You now understand:
- How volatility affects trading
- Bollinger Bands for overbought/oversold conditions
- ATR for volatility measurement and stop-loss placement
- Trading strategies combining BB and ATR

### Ready for Module 08?

**Next Module: Advanced Oscillators (Stochastic & Williams %R)**

In Module 08, you'll learn:
- Stochastic Oscillator (alternative to RSI)
- Williams %R indicator
- Divergence detection
- Combining multiple oscillators

**File**: `08_advanced_oscillators.ipynb`

---

### Quick Self-Assessment

Before moving on, can you answer these?

1. What do narrow Bollinger Bands indicate?
2. How do you calculate a 2 × ATR stop-loss?
3. When should you use bounce strategy vs breakout strategy?
4. What's the difference between ATR and Bollinger Bands?

**Answers**:
1. Low volatility, potential big move coming (squeeze)
2. Entry Price - (2 × ATR value)
3. Bounce for ranging markets, breakout for post-squeeze
4. ATR measures absolute volatility (RM), BB measures relative volatility (price extremes)

---

**Happy Trading! Remember: Practice with paper money first!**